In [1]:
# This file plays the mpc motion only in viz
%load_ext autoreload
%autoreload 2

import time
import numpy as np
import pinocchio as pin
import crocoddyl
from matplotlib import pyplot as plt

from py_biconvex_mpc.motion_planner.cpp_biconvex import BiConvexMP
from inverse_kinematics_cpp import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator
from robot_properties_solo.config import Solo12Config

import raisimpy as raisim
import subprocess
from py_biconvex_mpc.bullet_utils.solo_mpc_env import Solo12Env

from mpc_cartwheel import generate_plan

solo import 


pybullet build time: Jul  2 2021 21:43:18


In [2]:
robot = Solo12Config.buildRobotWrapper()
rmodel = robot.model
rdata = robot.data

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

pin_robot = Solo12Config.buildRobotWrapper()
pin_robot = Solo12Config.buildRobotWrapper()
urdf = Solo12Config.urdf_path


q0 = np.array(Solo12Config.initial_configuration)
v0 = pin.utils.zero(pin_robot.model.nv)
x0 = np.concatenate([q0, pin.utils.zero(pin_robot.model.nv)])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
viz.viewer.jupyter_cell()

In [47]:
sim_t = 0.0
sim_dt = .001
index = 0
pln_ctr = 0
plan_freq = 0.05 #0.05 # sec
update_time = 0.0 # sec (time of lag)
lag = int(update_time/sim_dt)

q = q0
v = v0

for o in range(10):
    contact_configuration = np.array([1,1,1,1])
    xs, us, f = generate_plan(q, v, sim_t)
    xs = xs[lag:]
    us = us[lag:]
    f = f[lag:]

    time.sleep(0.001)
    for ind in range(int(plan_freq/sim_dt)):
        viz.display(xs[ind][:robot.model.nq])
    
    q = xs[int(plan_freq/sim_dt)-1][0:pin_robot.model.nq]
    v = xs[int(plan_freq/sim_dt)-1][pin_robot.model.nq:]

    sim_t += plan_freq


[0.2        0.00038689 0.22596527 0.         0.         0.
 0.         0.         0.        ]
Maximum iterations reached 
Final norm: 0.00562343


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

[ 2.06348177e-01  3.39229031e-04  2.01578223e-01  1.41735964e-01
 -1.06473191e-03 -4.83345294e-01 -1.33919125e-05 -5.33958696e-02
 -1.63416314e-03]
breaking outer loop due to norm ...
Maximum iterations reached 
Final norm: 0.000943509


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

[ 2.21381272e-01  2.78679970e-04  1.90423200e-01  3.04981137e-01
 -1.38261310e-03 -2.17685527e-01  8.57519469e-05 -1.93065361e-02
 -7.47067229e-04]
Maximum iterations reached 
Final norm: 0.0206032


...

Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

[ 0.43841179 -0.00096579  0.25510943  0.80280649  0.00163336 -0.21334039
  0.00095615 -0.29444172 -0.0208423 ]
Maximum iterations reached 
Final norm: 0.00568179


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

[0.45512238 0.01116931 0.25462165 0.46051316 0.04105815 0.3828771
 0.03098911 0.92598978 0.16390658]
Maximum iterations reached 
Final norm: 0.00312068


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit